# PfDA, Ch 7: Data Cleaning and Preparation

Chapter 7 methods:
* Filter out missing data
* Fill in missing data
* Remove duplicates
* Transform data using either mapping or a function
* Replace values
* Discretization and Binning
* Manipulate Strings

In [1]:
import pandas as pd
import numpy as np
import re

## 7.1 Handling Missing Data

In [2]:
stringData = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [3]:
stringData

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
stringData.isnull()

0    False
1    False
2     True
3    False
dtype: bool

`None` is treated as `NaN`:

In [5]:
stringData[0] = None

In [6]:
stringData.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

Instead of `isnull` and boolean indexing, can use `dropna`:

In [7]:
from numpy import nan as NA

In [8]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [9]:
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [10]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is equivalent to:

In [11]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In a DF, dropna() by default drops *any* row containing a null value:

In [12]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], 
                     [NA, NA, NA], [NA, 6.5, 3.]])

In [13]:
cleaned = data.dropna()

In [14]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [15]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Drop only rows that are entirely NA by passing `how='all'`:

In [16]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Do the same with columns by passing `axis=1`:

In [17]:
data[4] = NA

In [18]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [19]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Set a threshold for minimum # of observations with `thresh`:

In [20]:
df = pd.DataFrame(np.random.randn(7, 3))

In [21]:
df.iloc[:4, 1] = NA

In [22]:
df.iloc[:2, 2] = NA

In [23]:
df

,0,1,2
0,-0.173926,NaN,NaN
1,1.178578,NaN,NaN
2,-0.695190,NaN,-1.146423
3,-0.234746,NaN,-0.100646
4,0.390061,0.448307,-0.509468
5,3.107572,-0.577096,-1.490313
6,0.249550,-0.420668,-0.622450


In [24]:
df.dropna()

,0,1,2
4,0.390061,0.448307,-0.509468
5,3.107572,-0.577096,-1.490313
6,0.249550,-0.420668,-0.622450


In [25]:
df.dropna(thresh=2) # rows with at least 2 observations

,0,1,2
2,-0.695190,NaN,-1.146423
3,-0.234746,NaN,-0.100646
4,0.390061,0.448307,-0.509468
5,3.107572,-0.577096,-1.490313
6,0.249550,-0.420668,-0.622450


### Filling In Missing Data

Use `fillna()` with a value to replace NaNs:

In [26]:
df.fillna(0)

,0,1,2
0,-0.173926,0.000000,0.000000
1,1.178578,0.000000,0.000000
2,-0.695190,0.000000,-1.146423
3,-0.234746,0.000000,-0.100646
4,0.390061,0.448307,-0.509468
5,3.107572,-0.577096,-1.490313
6,0.249550,-0.420668,-0.622450


Get different fill values for each column by passing in a dict:

In [27]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.173926,0.500000,0.000000
1,1.178578,0.500000,0.000000
2,-0.695190,0.500000,-1.146423
3,-0.234746,0.500000,-0.100646
4,0.390061,0.448307,-0.509468
5,3.107572,-0.577096,-1.490313
6,0.249550,-0.420668,-0.622450


Out of curiosity...

In [28]:
df.fillna({1: df[1].mean(), 2: df[2].median()})

,0,1,2
0,-0.173926,-0.183152,-0.622450
1,1.178578,-0.183152,-0.622450
2,-0.695190,-0.183152,-1.146423
3,-0.234746,-0.183152,-0.100646
4,0.390061,0.448307,-0.509468
5,3.107572,-0.577096,-1.490313
6,0.249550,-0.420668,-0.622450


Cool.

You can use `inplace=True` to modify the existing object:

In [29]:
df.fillna(0, inplace=True)

In [30]:
# Book has this:
_ = df.fillna(0, inplace=True)

In [31]:
df

,0,1,2
0,-0.173926,0.000000,0.000000
1,1.178578,0.000000,0.000000
2,-0.695190,0.000000,-1.146423
3,-0.234746,0.000000,-0.100646
4,0.390061,0.448307,-0.509468
5,3.107572,-0.577096,-1.490313
6,0.249550,-0.420668,-0.622450


Reindexing methods can also be used:

In [32]:
df = pd.DataFrame(np.random.randn(6, 3))

In [33]:
df.iloc[2:, 1] = NA

In [34]:
df.iloc[4:, 2] = NA

In [35]:
df

,0,1,2
0,-0.413550,-1.364119,0.739923
1,0.662479,0.102254,1.065181
2,-0.899570,NaN,-0.923365
3,0.479756,NaN,0.853614
4,-0.535689,NaN,NaN
5,0.782906,NaN,NaN


In [36]:
df.fillna(method='ffill')

,0,1,2
0,-0.413550,-1.364119,0.739923
1,0.662479,0.102254,1.065181
2,-0.899570,0.102254,-0.923365
3,0.479756,0.102254,0.853614
4,-0.535689,0.102254,0.853614
5,0.782906,0.102254,0.853614


In [37]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.413550,-1.364119,0.739923
1,0.662479,0.102254,1.065181
2,-0.899570,0.102254,-0.923365
3,0.479756,0.102254,0.853614
4,-0.535689,NaN,0.853614
5,0.782906,NaN,0.853614


You can even pass the mean value of a Series:

In [38]:
data = pd.Series([1., NA, 3.5, NA, 7])

In [39]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2 Data Transformation

### Removing Duplicates

In [40]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})

In [41]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [42]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [43]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Can drop duplicates on just a subset of columns:

In [44]:
data['v1'] = range(7)

In [45]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [46]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


Pass `keep='last'` to keep the last value instead of the first:

In [47]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [48]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 
                              'Pastrami', 'corned beef', 'Bacon', 
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [49]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Say you want to add a column by mapping what meat goes to what animal:

In [50]:
meatToAnimal = {
    'bacon': 'pig', 
    'pulled pork': 'pig', 
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

Use the `map` method on a Series:

In [51]:
# Inconsistent capitalization on meats; let's fix that first:
lowercased = data['food'].str.lower()

In [52]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [53]:
data['animal'] = lowercased.map(meatToAnimal)

Mapped the animal to the meat without changing the DF.

In [54]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


Alternatively, pass a function that does all the work:

In [55]:
data['food'].map(lambda x: meatToAnimal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [56]:
meatToAnimal['bacon']

'pig'

Fancy.

### Replacing Values

`replace` is simpler and more flexible than `fillna` or `map`:

In [57]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [58]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The **-999** might be a sentinel value for some missing data.  
Replace these with NA values.

In [59]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

Replace multiple values at once by passing a list:

In [60]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

Pass in a list of substitutes to replace different values for each:

In [61]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

Or try it as a dict:

In [62]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

Let's try this too:

In [63]:
replacements = {-999: np.nan, -1000: 0}

In [64]:
replacements

{-999: nan, -1000: 0}

In [65]:
data.replace(replacements)

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [66]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                   index=['Ohio', 'Colorado', 'New York'],
                   columns=['one', 'two', 'three', 'four'])

In [67]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [68]:
data.index

Index(['Ohio', 'Colorado', 'New York'], dtype='object')

In [69]:
transform = lambda x: x[:4].upper()

In [70]:
'colorado'[:4]

'colo'

In [71]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

Use `.index` to modify the DF in-place:

In [72]:
data.index = data.index.map(transform)

In [73]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Use `.rename` if you *don't* want to modify the original:

In [74]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Use `.rename` with a dict to rename just a subset of indices:

In [75]:
data.rename(index={'OHIO': 'INDIANA'},
           columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Use `inplace=True` to modify in-place:

In [76]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Let's try something...

In [77]:
testDict = {'OHIO': 'INDIANA'}

In [78]:
testDict['OHIO'].title()

'Indiana'

### Discretization and Binning

In [79]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Bin data using pandas `cut` function:

In [80]:
bins = [18, 25, 35, 60, 100]

In [81]:
cats = pd.cut(ages, bins)

This returns a **Categorical** object:

In [82]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [83]:
type(cats)

pandas.core.arrays.categorical.Categorical

In [84]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [85]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

Get the bin counts:

In [86]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

Note: ( indicates open, ] indicates closed (inclusive); these can be switched passing `right=False`:

In [87]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

Pass your own bin names using `labels=`:

In [88]:
groupNames = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [89]:
pd.cut(ages, bins, labels=groupNames)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

Let's try this...

In [90]:
pd.value_counts(pd.cut(ages, bins, labels=groupNames))

Youth         5
MiddleAged    3
YoungAdult    3
Senior        1
dtype: int64

In [91]:
data = np.random.rand(20)

Provide a number of (equal length) bins instead:

In [92]:
pd.cut(data, 4, precision=2) # precision=2 limits to 2 decimal points

[(0.26, 0.5], (0.74, 0.98], (0.74, 0.98], (0.74, 0.98], (0.74, 0.98], ..., (0.26, 0.5], (0.26, 0.5], (0.74, 0.98], (0.26, 0.5], (0.018, 0.26]]
Length: 20
Categories (4, interval[float64]): [(0.018, 0.26] < (0.26, 0.5] < (0.5, 0.74] < (0.74, 0.98]]

In [93]:
pd.value_counts(pd.cut(data, 4, precision=2))

(0.74, 0.98]     8
(0.26, 0.5]      8
(0.5, 0.74]      2
(0.018, 0.26]    2
dtype: int64

Cut into quantiles using `qcut` to get bins with approx. equal numbers of values:

In [94]:
data = np.random.randn(1000) # Normally distributed

In [95]:
cats = pd.qcut(data, 4) # Cut into quartiles

In [96]:
cats

[(-0.707, -0.0302], (-3.108, -0.707], (-0.0302, 0.648], (0.648, 3.061], (-3.108, -0.707], ..., (0.648, 3.061], (-3.108, -0.707], (-0.707, -0.0302], (0.648, 3.061], (0.648, 3.061]]
Length: 1000
Categories (4, interval[float64]): [(-3.108, -0.707] < (-0.707, -0.0302] < (-0.0302, 0.648] < (0.648, 3.061]]

In [97]:
pd.value_counts(cats)

(0.648, 3.061]       250
(-0.0302, 0.648]     250
(-0.707, -0.0302]    250
(-3.108, -0.707]     250
dtype: int64

Nice.

You can even pass your own quantiles (between 0 and 1, inclusive):

In [98]:
cats = pd.qcut(data, [0, .1, .5, .9, 1])

In [99]:
pd.value_counts(cats)

(-0.0302, 1.322]     400
(-1.324, -0.0302]    400
(1.322, 3.061]       100
(-3.108, -1.324]     100
dtype: int64

### Detecting and Filtering Outliers

In [100]:
data = pd.DataFrame(np.random.randn(1000, 4)) # 4 columns, each with 1000 rows; normally distributed

In [101]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.011839,0.041994,0.014533,-0.042362
std,0.987798,0.999527,0.997615,0.998559
min,-3.889328,-3.069107,-2.927182,-3.597200
25%,-0.647521,-0.684857,-0.648505,-0.738672
50%,-0.031912,0.060737,0.024103,-0.040545
75%,0.624014,0.712489,0.696021,0.637784
max,3.616530,3.127403,3.708164,3.195272


Find the values from one column which exceed |3|:

In [102]:
col = data[2]

In [103]:
col[np.abs(col) > 3]

182    3.708164
244    3.634789
596    3.439703
Name: 2, dtype: float64

Use the `any` method to select all the rows with any values exceeding |3|:

In [104]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
34,0.649112,-0.702071,0.704326,3.195272
121,3.616530,-1.242099,-0.124133,-0.535412
182,-0.893231,1.459472,3.708164,-0.327924
208,0.667002,-3.069107,-0.368250,-0.848700
213,-3.889328,0.179762,-1.168038,1.831309
244,0.444855,0.945563,3.634789,-0.912806
375,0.209303,3.127403,0.990445,0.652916
418,0.508058,-1.856918,-0.171430,-3.597200
596,-0.456824,-0.436301,3.439703,2.481188


Cap the values outside |3| to -3 or 3:

In [105]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [106]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.011567,0.041936,0.012750,-0.041960
std,0.982617,0.998929,0.991705,0.995992
min,-3.000000,-3.000000,-2.927182,-3.000000
25%,-0.647521,-0.684857,-0.648505,-0.738672
50%,-0.031912,0.060737,0.024103,-0.040545
75%,0.624014,0.712489,0.696021,0.637784
max,3.000000,3.000000,3.000000,3.000000


Note: `.sign()` produces a '-1' or a '1' depending on the sign of the value:

In [107]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,1.0,1.0
1,-1.0,1.0,-1.0,1.0
2,1.0,-1.0,1.0,1.0
3,-1.0,-1.0,1.0,1.0
4,1.0,-1.0,-1.0,-1.0


Neat.

### Permutation and Random Sampling

In [108]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


Use `.permutation()` with the length of the axis to produce an array of integers for the new ordering:

In [109]:
sampler = np.random.permutation(5)
sampler

array([1, 2, 3, 4, 0])

Use `take` function (equivalent to `iloc`-based indexing):

In [110]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
0,0,1,2,3


Use the `sample` method to select a random subset without replacement:

In [111]:
df.sample(n=3)

,0,1,2,3
1,4,5,6,7
4,16,17,18,19
2,8,9,10,11


Sample *with* replacement by passing `replace=True`:

In [112]:
choices = pd.Series([5, 7, -1, 6, 4])
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [113]:
draws = choices.sample(n=10, replace=True)
draws

0    5
4    4
0    5
0    5
0    5
1    7
0    5
3    6
1    7
4    4
dtype: int64

### Computing Indicator/Dummy Variables

In [114]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                  'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


Use the `get_dummies` function to derive a dummy matrix:

In [115]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


You can add a prefix to the columns using `prefix=`:

In [116]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [117]:
dfWithDummies = df[['data1']].join(dummies)
dfWithDummies

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


What if a row belongs to multiple categories, like movie genres?

In [118]:
mnames = ['movie_id', 'title', 'genres']

In [119]:
movies = pd.read_table('movies.dat', sep='::', header=None, names=mnames, engine='python')
movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


Time to do some data wranglin'  
First, extract the genres in the dataset:

In [120]:
movies.genres[1].split('|')

['Adventure', "Children's", 'Fantasy']

In [121]:
allGenres = []
for x in movies.genres:
    allGenres.extend(x.split('|'))

For fun, let's see how many of each genre there are:

In [122]:
pd.value_counts(allGenres)

Drama          1603
Comedy         1200
Action          503
Thriller        492
Romance         471
Horror          343
Adventure       283
Sci-Fi          276
Children's      251
Crime           211
War             143
Documentary     127
Musical         114
Mystery         106
Animation       105
Fantasy          68
Western          68
Film-Noir        44
dtype: int64

Then get the unique genres:

In [123]:
genres = pd.unique(allGenres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

One way to construct an indicator DF is to start with a DF of all zeroes:

In [124]:
zeroMatrix = np.zeros((len(movies), len(genres)))
zeroMatrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [125]:
dummies = pd.DataFrame(zeroMatrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Iterate through each movie and set entries in each row of **dummies** to 1, using `dummies.columns` to compute column indices:

In [126]:
gen = movies.genres[0]
gen

"Animation|Children's|Comedy"

In [127]:
gen.split('|')

['Animation', "Children's", 'Comedy']

In [128]:
dummies.columns.get_indexer(gen.split('|')) # Provides the indices

array([0, 1, 2], dtype=int64)

Now use `iloc` to set values based on these indices:

In [129]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [130]:
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now combine this back with **movies**:

In [131]:
moviesGenInd = movies.join(dummies.add_prefix('Genre_'))
moviesGenInd.head()

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
moviesGenInd.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

For statistical applications, can combine `get_dummies` with a discretionary function like `cut`:

In [133]:
np.random.seed(42)

In [134]:
values = np.random.rand(10)
values

array([0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864,
       0.15599452, 0.05808361, 0.86617615, 0.60111501, 0.70807258])

In [135]:
bins = [0, 0.2, .4, .6, .8, 1]

In [136]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,1,0,0,0
1,0,0,0,0,1
2,0,0,0,1,0
3,0,0,1,0,0
4,1,0,0,0,0
5,1,0,0,0,0
6,1,0,0,0,0
7,0,0,0,0,1
8,0,0,0,1,0
9,0,0,0,1,0


## 7.3 String Manipulation

### String Object Methods

A comma-separated string can be broken into pieces with `.split()`

In [137]:
val = 'a,b,  guido'

In [138]:
val.split(',')

['a', 'b', '  guido']

Use with `.strip()` to strip whitespace:

In [139]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

You *could* concatenate these pieces with a two-colon delimiter like this:

In [140]:
first, second, third = pieces

In [141]:
first + '::' + second + '::' + third

'a::b::guido'

OR you could use the '::' string and the `.join()` method:

In [142]:
'::'.join(pieces)

'a::b::guido'

The best way to detect a substring is using `in`, though `.index()` and `.find()` can also be used:

In [143]:
'guido' in val

True

In [144]:
val.index(',')

1

Note: `.index()` will return an error if the value isn't found, while `.find()` returns '-1':

In [145]:
# val.index(':')  # Produces an error

In [146]:
val.find(':')

-1

`.count()` will return the number of occurences:

In [147]:
val.count(',')

2

`.replace()` will substitute occurrences of one pattern for another:  
(will also delete patterns by passing an empty string)

In [148]:
val.replace(',', '::')

'a::b::  guido'

In [149]:
val.replace(',', '')

'ab  guido'

In [150]:
val.replace(' ', '')

'a,b,guido'

In [151]:
val.replace(',', '::').replace(' ', '')

'a::b::guido'

### Regular Expressions

Split a string with variable # of whitespace chars (tab, space, newline) using `\s+`:

In [152]:
import re

In [153]:
text = 'foo    bar\t baz  \tqux'

In [156]:
print(text)

foo    bar	 baz  	qux


In [154]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

Compile the *regex* yourself using `re.compile()`:

In [157]:
regex = re.compile('\s+')

In [158]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

Show a list of all patterns matching the regex using `.findall()` method:

In [159]:
regex.findall(text)

['    ', '\t ', '  \t']

Related to `.findall()`, `.search()` will return only the first result, and `.match()` only matches at the beginning of a string.

In [160]:
text = """
Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [161]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [162]:
# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

Using `.findall()` on the text produces a list of the email addresses:

In [163]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`.search()` returns the first email in the list:

In [166]:
m = regex.search(text)
m

<re.Match object; span=(6, 21), match='dave@google.com'>

In [168]:
text[m.start():m.end()]

'dave@google.com'

In [170]:
m.start(), m.end()

(6, 21)

`regex.match` only matches if the pattern occurs at the start of the string:

In [173]:
print(regex.match(text))

None


`.sub()` will return a string with occurrences replaced:

In [175]:
print(regex.sub('REDACTED', text))


Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



If you want to separate the components, add parentheses around the parts to segment:

In [178]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

A match object produced by this returns a tuple of the pattern components with `.group()` method:

In [180]:
m = regex.match('wesm@bright.net')

In [182]:
m.groups()

('wesm', 'bright', 'net')

`.findall()` returns a list of tuples when the pattern has groups:

In [183]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

`.sub()` also has access to groups. \1 corresponds to the 1st matched group, etc:

In [184]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))


Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com

